## Read Airspeed

Uses model to open and interpret vertical speed readings

TODO: Combine this file back into read_vertical_speed - just make it configurable

## Imports

In [ ]:
import cv2
import numpy as np
import tensorflow as tf

## Read Vertical Speed

Currently only works on a single file

In [ ]:
# Load the trained model (replace 'model.h5' with the path to your .h5 model file)
model = tf.keras.models.load_model('../models/airspeed_model.h5')

# Function to preprocess an input image
def preprocess_image(image_path):
    # Load and preprocess the image
    image = cv2.imread(image_path)
    image = image / 255.0  # Normalize pixel values to [0, 1]
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

def preprocess_frame(frame):
    
    frame = frame / 255.0  # Normalize pixel values to [0, 1]
    frame = np.expand_dims(frame, axis=0)  # Add batch dimension
    return frame



# Function to predict the vertical speed
def predict_airspeed(image_path):
    # Preprocess the input image
    preprocessed_image = preprocess_image(image_path)

    # Make a prediction using the model
    predictions = model.predict(preprocessed_image)

    # Convert the prediction to a vertical speed value (you may need to adjust this based on your label encoding)
    predicted_speed = np.argmax(predictions, axis=1)  # Assuming one-hot encoding

    return predicted_speed[0]

# Function to predict the vertical speed
def predict_airspeed_frame(frame):
    # Preprocess the input image
    preprocessed_image = preprocess_frame(frame)

    # Make a prediction using the model
    predictions = model.predict(preprocessed_image)

    # Convert the prediction to a vertical speed value (you may need to adjust this based on your label encoding)
    predicted_speed = np.argmax(predictions, axis=1)  # Assuming one-hot encoding

    return (predicted_speed[0] * 10)


In [ ]:
# Hacky means of making sure we don't go into infinite loop
def exit_application():
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.waitKey(1)
    cv2.waitKey(1)

In [ ]:
input_file_path = '../media/input_video/daytime_fixed_A10C_original_demo.mp4'

# Coordinates for vertical speed indicator
start_x = 750
start_y = 770

width = 120
height = 110

overlay_x = 200
overlay_y = 200

scale_factor = 3

cap = cv2.VideoCapture(input_file_path)

while True:
    ret, frame = cap.read()

    if not ret:
        break  # Break the loop if no more frames are available

    x, y, w, h = (start_x, start_y, width, height)
    cropped_frame = frame[y:y+h, x:x+w]

    resized_cropped_frame = cv2.resize(cropped_frame, None, fx=scale_factor, fy=scale_factor)

    frame[overlay_y:overlay_y+(h*scale_factor), overlay_x:overlay_x+(w*scale_factor)] = resized_cropped_frame

    vertical_speed = predict_airspeed_frame(cropped_frame)

    print(vertical_speed)

    cv2.putText(frame, f'Airspeed: {vertical_speed}', (150, 200), cv2.FONT_HERSHEY_PLAIN, 4.0, (0, 255, 0), 2)


    # Display the current frame
    cv2.imshow('Frame', frame)

    # Wait for user input (e.g., a key press)
    key = cv2.waitKey(0)

    # Check if the user pressed the 'q' key to exit
    if key == ord('q'):
        break

# Release the video capture object and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()
exit_application()